<a href="https://colab.research.google.com/github/thedeenun/NaiveBayes-LinearRegression-python-from-scratch/blob/main/NaiveBayes-LinearRegression-python-from-scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#รายชื่อสมาชิก
1. นายชิษณุพงษ์ 	เริงหทัยธรรม 	6209650263 
2. นายสินชู 	โชติกเสถียร 	6209650289
3. นายอาดีนัน 	อับดุลลี 		6209650719

In [ ]:
import numpy as np
import pandas as pd
import math
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

Import Dataset ทั้ง 3 dataset
1. iris dataset
2. red wine dataset
3. loan dataset

In [ ]:
iris = pd.read_csv('/content/iris.csv')
redwine = pd.read_csv('/content/winequality-red.csv')
loan = pd.read_csv('/content/train_loan.csv')

# การบ้านที่ 3 ข้อที่ 2

## ข้อที่ 2.1 Create Model

### Naive Bayes

In [ ]:
### ---------------------- ข้อ 2.1 ------------------------- ###
class NaiveBayesClassifier():
    '''
    สูตร Naive Bayes 
    P(y|X) = P(X|y) * P(y) / P(X)
    '''
    def cal_statistics(self, features, target):
        '''
        คำนวณค่า mean, variance ของแต่ละคอลัมน์
        ''' 
        self.mean = features.groupby(target).apply(np.mean).to_numpy()
        self.var = features.groupby(target).apply(np.var).to_numpy()
              
        return self.mean, self.var

    def cal_prior(self, features, target):
        '''
        คำนวณค่า prior probability P(y)
        '''
        self.prior = (features.groupby(target).apply(lambda x: len(x)) / self.rows).to_numpy()

        return self.prior

    def gaussian(self, class_idx, x):     
        '''
        เนื่องข้อมูลที่จะนำมาใช้เป็น continuous values จึงต้อง Gaussian density function ในการคำนวณ
        (1/√2pi*σ) * exp((-1/2)*((x-μ)^2)/(2*σ²))
        '''
        mean = self.mean[class_idx]
        var = self.var[class_idx]
        numerator = np.exp((-1/2)*((x-mean)**2) / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        prob = numerator / denominator
        return prob   

    def cal_posterior(self, x):
        '''
        คำนวณค่า posteriors probability P(X|y)
        '''
        posteriors = []
        
        for i in range(self.count):
            prior = np.log(self.prior[i])
            conditional = np.sum(np.log(self.gaussian(i, x)))
            posterior = prior + conditional
            posteriors.append(posterior)
        
        return self.classes[np.argmax(posteriors)]

    #ฟังก์ชั่น Fit หรือ เทรนโมเดล จะไปเรียกใช้ฟังก์ชันอื่น ๆ ในโมเดล
    def fit(self, features, target):
        self.classes = np.unique(target)
        self.count = len(self.classes)
        self.feature_nums = features.shape[1]
        self.rows = features.shape[0]
        
        self.cal_statistics(features, target)
        self.cal_prior(features, target)
    
    #ฟังก์ชั่น predict หรือ ทำนาย 
    def predict(self, features):
        preds = [self.cal_posterior(f) for f in features.to_numpy()]
        return preds

    #ฟังก์ชั่น accuracy ในการวัดค่าความแม่นยำ
    def accuracy(self, y_test, y_pred):
        accuracy = np.sum(y_test == y_pred) / len(y_test)#นำผลรวมของค่าที่เหมือนกันจากตัวแปร y_test ที่เป็นค่าจริง และ y_pred เป็นค่าที่ได้จากการทำนายหารกับจำนวนค่าทั้งหมดที่อยู๋ใน y_test
        return accuracy #return ผลลัพธ์ออกเป็นค่า accuracy ที่เป็ฯค่าความแม่นยำของการทำนายเมื่อเทียบกับค่าจริงที่มีอยู่แล้ว

### Linear Regression

In [ ]:
### ---------------------- ข้อ 2.1 ------------------------- ###
class LinearRegression():

    def __init__(self, learning_rate=0.0001, n_iterations=1000):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.theta = None
        self.loss = []
        self.rmse = []
        self.mse = []
    
    #ฟังก์ชั่น Fit หรือ เทรนโมเดล ที่มีการใช้ Gradient descent และ อัปเดตค่า theta
    def fit(self, X, y):
        
        #กำหนดค่า theta แรกเริ่มให้เป็น 0
        self.theta = np.zeros(X.shape[1])
     
        #Gradient descent
        for i in range(self.n_iterations):
          y_hat = np.dot(X, self.theta)
          j = (1/(2*X.shape[0]))*np.sum(np.square(y_hat - y))
          self.loss.append(j)#เก็บค่า loss/error สำหรับแต่ละ iteration

          #เก็บค่า RMSE สำหรับแต่ละ iteration
          rmse_loss = self.r_mean_squared_error(y, y_hat)
          self.rmse.append(rmse_loss)

          #เก็บค่า MSE สำหรับแต่ละ iteration
          mse_loss = self.mean_squared_error(y, y_hat)
          self.mse.append(mse_loss)

          #Partial derivatives และ การอัปเดตค่า theta
          d_theta = (1/X.shape[0]) * (2* np.dot(X.T, y_hat - y))
          self.theta -= self.learning_rate * d_theta

    #ฟังก์ชั่น predict หรือ ทำนายโดยนำ feature ของชุดข้อมูลมาคูณกับค่า theta ที่ได้จากการเทรน
    def predict(self, X):
        return np.dot(X, self.theta)

        #ฟังก์ชั่น Root mean squared error   
    def r_mean_squared_error(self, y, y_hat):
        error = 0
        for i in range(len(y)):
            error += (y[i] - y_hat[i]) ** 2
        return np.sqrt(error/len(y))

    #ฟังก์ชั่น Mean squared error
    def mean_squared_error(self, y, y_hat):
        error = 0
        for i in range(len(y)):
            error += (y[i] - y_hat[i]) ** 2
        return error / len(y)

    ### ------------------------ ข้อ 2.4 ------------------------- ###

    #การเปรียบเทียบค่า RMSE กับ training set ในแต่ละ iteration ข้อ 2.4.1
    def iteration_com_train(self, X, y, iteration):
        self.n_iterations = iteration
        self.fit(X, y)
        for i in range(self.n_iterations):
          if(i%(self.n_iterations/10) == 0):
            print("iteration ",i," error: ", self.loss[i]," and rmse: ", self.rmse[i])
    
    #การเปรียบเทียบค่า RMSE กับ test set ในแต่ละ iteration ข้อ 2.4.1
    def iteration_com_test(self, X_test, y_test, iteration):
        self.n_iterations = iteration
        for i in range(self.n_iterations):
          y_hat = self.predict(X_test)
          if(i%(self.n_iterations/10) == 0):
            print("iteration ",i," error: ", self.loss[i]," and rmse: ", self.r_mean_squared_error(y_test, y_hat))

    #การเปรียบเทียบค่า MSE กับ training set เมื่อเปลี่ยนแปลงค่า learning rate ข้อ 2.4.2
    def learning_rate_com(self, X, y, learning_rate):
        self.learning_rate = learning_rate
        self.fit(X, y)
        print("learning rate: ",self.learning_rate ," MSE: ", self.mse[-1])


## ข้อที่ 2.2 Train and Test

### A.(iris flower)

แบ่ง feature กับ output(target) ไว้อยู่ในตัวแปร iris_X และ iris_y ตามลำดับ

จากนั้นใช้ฟังก์ชั่น train_test_split ในการแบ่ง data เป็น training set กับ test set เป็น 80:20

In [ ]:
iris_X, iris_y = iris.iloc[:, :-1], iris.iloc[:, -1]
iris_X_train, iris_X_test, iris_y_train, iris_y_test = train_test_split(iris_X, iris_y, test_size=0.2)

เรียกใช้โมเดล NaiveBayesClassifier() ไว้ในตัวแปร nb_model

เรียกใช้ฟังก์ชั่น fit ในโมเดลที่สร้างไว้เพื่อทำการ train โมเดลโดยใช้ iris_X_train เป็นข้อมูลสอน และ iris_y_train เพื่อเป็นข้อมูลทวนการสอนของโมเดล

In [ ]:
nb_model = NaiveBayesClassifier()
nb_model.fit(iris_X_train, iris_y_train)

จากนั้นให้โมเดล nb_model ทำการทำนายโดยเรียกใช้ฟังก์ชั่น predict เพื่อทำนายข้อมูลทดสอบใน iris_X_test และเก็บผลลัพธ์ที่ทำนายได้ใน iris_pred

In [ ]:
iris_pred = nb_model.predict(iris_X_test)

นำผลลัพธ์จากการทำนายที่อยู่ใน iris_pred มาเทียบกับข้อมูลเฉลยที่มีอยู่แล้วจาก iris_y_test และแสดงผลลัพธ์ที่เป็นค่าความแม่นยำออกมาตามผลลัพธ์ที่ปรากฎ

In [ ]:
print("Accuracy for Iris: %.4f" % nb_model.accuracy(iris_y_test, iris_pred))

Accuracy for Iris: 0.9333


### B.(Red wine)

แบ่ง feature กับ output(target) ไว้อยู่ในตัวแปร redwine_X และ redwine_y ตามลำดับ

จากนั้นใช้ฟังก์ชั่น train_test_split ในการแบ่ง data เป็น training set กับ test set เป็น 80:20

In [ ]:
redwine_X, redwine_y = redwine.iloc[:, :-1].values, redwine.iloc[:, -1].values
redwine_X_train, redwine_X_test, redwine_y_train, redwine_y_test = train_test_split(redwine_X, redwine_y, test_size=0.2)

เรียกใช้โมเดล LinearRegression() ไว้ในตัวแปร lr_model

เรียกใช้ฟังก์ชั่น fit ในโมเดลที่สร้างไว้เพื่อทำการ train โมเดลโดยใช้ redwine_X_train เป็นข้อมูลสอน และ redwine_y_train เพื่อเป็นข้อมูลทวนการสอนของโมเดล

In [ ]:
lr_model = LinearRegression()
lr_model.fit(redwine_X_train, redwine_y_train)

จากนั้นให้โมเดล lr_model ทำการทำนายโดยเรียกใช้ฟังก์ชั่น predict เพื่อทำนายข้อมูลทดสอบใน redwine_X_test และเก็บผลลัพธ์ที่ทำนายได้ใน redwine_pred

In [ ]:
redwine_pred = lr_model.predict(redwine_X_test)

นำผลลัพธ์จากการทำนายที่อยู่ใน redwine_pred มาเทียบกับข้อมูลเฉลยที่มีอยู่แล้วจาก redwine_y_test และแสดงผลลัพธ์ที่เป็นค่า Root mean squared error ออกมาตามผลลัพธ์ที่ปรากฎ

In [ ]:
print("RMSE: ", lr_model.r_mean_squared_error(redwine_y_test, redwine_pred))
print("MSE: ", lr_model.mean_squared_error(redwine_y_test, redwine_pred))

RMSE:  0.7322613899295386
MSE:  0.5362067431815397


### C.(Loan prediction)

ในส่วนนี้เป็นขั้นตอนการเตรียมข้อมูลสำหรับชุดข้อมูล Loan
1. ทำการดรอปคอลัมน์ Lona_ID ทิ้ง
2. ดรอปแถวที่มีค่า Null ทิ้ง
3. ทำ One hot encoding เพื่อแปลงข้อมูลที่เป็นประเภท object หรือ string ในอยู่ในรูปแบบ 1 กับ 0 เนื่องจากข้อมูลบางอย่างเป็น Discrete ให้เป็นค่า Continuous

In [ ]:
loan = loan.drop(columns=['Loan_ID'])#1
loan.dropna(axis=0, inplace=True)#2
loan = pd.get_dummies(loan, columns = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area'])#3
loan = pd.DataFrame(loan)

แบ่ง feature กับ output(target) ไว้อยู่ในตัวแปร loan_X และ loan_y ตามลำดับ

จากนั้นใช้ฟังก์ชั่น train_test_split ในการแบ่ง data เป็น training set กับ test set เป็น 80:20

In [ ]:
loan_X, loan_y = loan.drop(columns='Loan_Status'), loan.loc[:,'Loan_Status']
loan_X_train, loan_X_test, loan_y_train, loan_y_test = train_test_split(loan_X, loan_y, test_size=0.2)

เรียกใช้โมเดล NaiveBayesClassifier() ไว้ในตัวแปร nb_model

เรียกใช้ฟังก์ชั่น fit ในโมเดลที่สร้างไว้เพื่อทำการ train โมเดลโดยใช้ loan_X_train เป็นข้อมูลสอน และ loan_y_train เพื่อเป็นข้อมูลทวนการสอนของโมเดล

In [ ]:
nb_model = NaiveBayesClassifier()
nb_model.fit(loan_X_train, loan_y_train)

จากนั้นให้โมเดล nb_model ทำการทำนายโดยเรียกใช้ฟังก์ชั่น predict เพื่อทำนายข้อมูลทดสอบใน loan_X_test และเก็บผลลัพธ์ที่ทำนายได้ใน loan_pred

In [ ]:
loan_pred = nb_model.predict(loan_X_test)

นำผลลัพธ์จากการทำนายที่อยู่ใน loan_pred มาเทียบกับข้อมูลเฉลยที่มีอยู่แล้วจาก loan_y_test และแสดงผลลัพธ์ที่เป็นค่าความแม่นยำออกมาตามผลลัพธ์ที่ปรากฎ

In [ ]:
print("Accuracy for Loan: %.4f" % nb_model.accuracy(loan_y_test, loan_pred))

Accuracy for Loan: 0.8333


## ข้อที่ 2.4 Evaluate Regression model

### ข้อที่ 2.4.1 เปรียบเทียบค่า RMSE เฉลี่ยของ Training set และ Test set สําหรับแต่ละ Iteration


ใช้โมเดล lr_model เรียกใช้ฟังก์ชัน iteration_com_train สำหรับเปรียบเที่ยบค่า RMSE เฉลี่ยของ training set ตามจำนวน iteration ที่กำหนดไว้ และแสดงผลลัพธ์ออกมาพร้อมกับค่า loss และ RMSE

ใช้โมเดล lr_model เรียกใช้ฟังก์ชัน iteration_com_test สำหรับเปรียบเที่ยบค่า RMSE เฉลี่ยของ test set ตามจำนวน iteration ที่กำหนดไว้ และแสดงผลลัพธ์ออกมาพร้อมกับค่า loss และ RMSE

In [ ]:
print("Training set:")
lr_model.iteration_com_train(redwine_X_train, redwine_y_train, iteration=1000)
print("\nTest set:")
lr_model.iteration_com_test(redwine_X_test, redwine_y_test, iteration=1000)

Training set:
iteration  0  error:  16.216575449569977  and rmse:  5.695011053469515
iteration  100  error:  0.6654564386745815  and rmse:  1.1536519741018796
iteration  200  error:  0.3390327860621484  and rmse:  0.8234473705855746
iteration  300  error:  0.29168380827693646  and rmse:  0.763785059132392
iteration  400  error:  0.2809623931449236  and rmse:  0.7496164261072775
iteration  500  error:  0.2766491731631976  and rmse:  0.7438402693632523
iteration  600  error:  0.2736907495292878  and rmse:  0.7398523494985844
iteration  700  error:  0.2711772949568382  and rmse:  0.7364472757188231
iteration  800  error:  0.26892815700215056  and rmse:  0.733386878805655
iteration  900  error:  0.26689375390949693  and rmse:  0.7306076291820344

Test set:
iteration  0  error:  16.216575449569977  and rmse:  0.7322613899295386
iteration  100  error:  0.6654564386745815  and rmse:  0.7322613899295386
iteration  200  error:  0.3390327860621484  and rmse:  0.7322613899295386
iteration  300  e

จากผลลัพธ์จะเห็นได้ว่า สำหรับชุดข้อมูล training set ค่า RMSE เฉลี่ยของแต่ละรอบจะยิ่งลดลงไปเรื่อย ๆ ตามจำนวน iteration ที่กำหนด

แต่สำหรับชุดข้อมูล test set ค่า RMSE เฉลี่ยของแต่ละรอบจะมีค่าเท่ากันในทุกรอบ เนื่องจากโมเดลจะทำการนำค่า y_hat คือผลลัพธ์ของการทดสอบมาเทียบกับค่าจริงที่อยู่ใน redwine_y_test และทุกครั้งที่นำมาเทียบค่าที่อยู๋ใน y_hat ที่ได้จากการทดสอบยังทดสอบได้ค่าเดิมเสมอ จึงทำให้ค่า RMSE มีค่าเท่ากันเสมอ

### ข้อที่ 2.4.2 เปรียบเทียบค่าของ Squared Errors ของ Training set เมื่อมีการเปลี่ยนแปลงค่า Learning rate

ใช้โมเดล lr_model เรียกใช้ฟังก์ชั่น learning_rate_com โดยการจูนพารามิเตอร์ learning_rate และแสดงผลลัพธ์เป็นค่า MSE (Squared error)

In [ ]:
lr_model.learning_rate_com(redwine_X_train, redwine_y_train, learning_rate=0.0001)

learning rate:  0.0001  MSE:  0.5301339834692355


In [ ]:
lr_model.learning_rate_com(redwine_X_train, redwine_y_train, learning_rate=0.00005)

learning rate:  5e-05  MSE:  0.5533460756698676
